## Tools 
Models can request to calls tools to perform tasks such as fetching data from a database or a web page, searching web, or running code.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
model.invoke("who is rohit sharma?") 

AIMessage(content='<think>\nOkay, the user is asking "who is rohit sharma?" Let me start by recalling what I know about him. He\'s a cricketer from India, right? I think he plays for the Indian cricket team. His full name is Rohit Sharma, and he\'s known for being a batsman. \n\nFirst, I should mention his birth details. I believe he was born in 1987 in Mumbai. That makes him around 36 years old now. He\'s a key player in the Indian team, maybe even their captain in some formats. Wait, he was the captain of the Indian ODI and T20I teams, but I think he stepped down recently. Virat Kohli took over as ODI captain, so maybe that\'s a point to include.\n\nHe\'s known for his aggressive batting style. I remember he\'s one of the highest run-scorers in international cricket. Let me check the stats. He holds the record for the most runs in ODIs and Test cricket. Oh, and he\'s a left-handed batsman. That\'s a unique trait because he\'s one of the few left-handers in the team. \n\nHe\'s also pl

In [2]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"The weather in {location} is sunny."

model_with_tools = model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What is the weather in Boston?")
print(response)
for tool_call in response.tool_calls:
    # view tool call made by model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")


content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified Boston, I need to call this function with "Boston" as the location. I\'ll make sure the parameters are correctly formatted in JSON. No other tools are provided, so this should be the only function call needed.\n', 'tool_calls': [{'id': 'mvqmjt7kb', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 153, 'total_tokens': 257, 'completion_time': 0.178116874, 'completion_tokens_details': {'reasoning_tokens': 80}, 'prompt_time': 0.007360807, 'prompt_tokens_details': None, 'queue_time': 0.058495413, 'total_time': 0.185477681}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model = "groq:qwen/qwen3-32b",
    tools = [get_weather],
    system_prompt = "You are a helpful assistant that can answer questions about the weather."
)
agent.invoke({"messages":"What is the weather like in New York?"})

{'messages': [HumanMessage(content='What is the weather like in New York?', additional_kwargs={}, response_metadata={}, id='156fee6a-e4ad-4673-b624-3fd722f94170'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in New York. Let me check if I have the right tool for that. The available function is get_weather, which requires a location parameter. New York is a valid location, so I should call that function. I need to make sure to format the tool call correctly within the XML tags. Let me structure the JSON with the function name and the location argument. That's it, I'll return the tool call as specified.\n", 'tool_calls': [{'id': 'cr47djmtx', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 168, 'total_tokens': 288, 'completion_time': 0.200973733, 'completion_tokens_details': {'reasoning_tokens': 95

### Tool Execution Loops

In [ ]:
# model genraters tool calls
messages = [{"role": "user", "content": "what is weather in Mumbai"}]
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)

# Execute toola and  collect results
for tool_call in ai_message.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.content)
